**Description:**
This notebook is designed to update a collection in MongoDB. The script connects to a MongoDB database, fetches a specific collection, and updates records based on a provided data set.

In [78]:
import json
import pymongo
from pymongo import MongoClient, UpdateOne
from bson.objectid import ObjectId

# Load the MongoDB connection string from the JSON file
PATH_TO_SECRET_JSON = '/home/jovyan/keys/mongodb_key.json'
with open(PATH_TO_SECRET_JSON) as f:
    MONGODB_URI = json.load(f)['connection_string']

# Extract the database name from the connection string
db_name = MONGODB_URI.split('/')[-1].split('?')[0]

# Create a MongoDB client using the connection string
if MONGODB_URI:
    client = MongoClient(MONGODB_URI)
    print("The MongoDB client has been initialized.")
else:
    print("Failed to initialize the MongoDB client.")

# Print the version of the pymongo package
pymongo_version = pymongo.__version__
print(f"The version of the pymongo package is {pymongo_version}")

# Connect to the specified MongoDB database and collection
db = client[db_name]
collection = db.attractions

The MongoDB client has been initialized.
The version of the pymongo package is 4.4.1


In [79]:
# Find the document with the specified ObjectId in the 'attractions' collection
document = collection.find_one({"_id": ObjectId("6450c55c3879cf70e53ff044")})

# Print the retrieved document
print(document)


{'_id': ObjectId('6450c55c3879cf70e53ff044'), 'title': 'Schermerhorn Building', 'key': 'SchermerhornBuilding', 'featureKey': 'landmark', 'catalog': 'nycwayfinding', 'loc': {'lat': 40.72761, 'lon': -73.99378, 'location': '376-380 Lafayette Street', 'neighborhood': 'Greenwich Village', 'postalCode': '10012', 'city': 'New York', 'state': 'NY'}, 'map': {'center': {'latitude': 40.72761, 'longitude': -73.99378}, 'zoom': 17}, 'machineTags': [{'tag': 'nycwayfinding:lp=00193'}], 'photo': {'_id': ObjectId('649340baf86d4f4d60e5f631'), 'photoId': 52920417345, 'title': 'Schermerhorn Building', 'url': 'https://live.staticflickr.com/65535/52920417345_6a30663600_t.jpg', 'width': 100, 'height': 75}, 'inventory': {'styles': ['N/A'], 'architects': ['Henry J. Hardenbergh']}, 'aliases': ['376-380 Lafayette Street Building']}


**Description:**
We have a dataset of attractions with specific fields to be updated in the database. Below is the dataset:

In [67]:
data = [
{"id": "646c1a8b9e866c1d68eb57b1", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
{"id": "646c2ba39e866c1d68eb57bc", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10038"},
{"id": "5541973b083cd88a44265cf6", "loc.boroughCode": "MN", "loc.neighborhood": "Midtown", "loc.postalCode": "10019"},
{"id": "646c28aa9e866c1d68eb57b7", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
{"id": "646c307f9e866c1d68eb57c0", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
{"id": "646c2b099e866c1d68eb57bb", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
{"id": "644cc0b53879cf70e53ff027", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
{"id": "646c311c9e866c1d68eb57c1", "loc.boroughCode": "MN", "loc.neighborhood": "Lower East Side", "loc.postalCode": "10002"},
]

**Description:**
Now, we'll loop through the dataset and update the records in the MongoDB collection.

In [70]:
# Initialize counters and operations list
totalRecords = len(data)
recordsUpdated = 0
operations = []

# Prepare bulk update operations for each record in data
for item in data:
    update_fields = {}

    # Iterate over all keys in the item
    for key, value in item.items():
        # Check if the value is a dictionary (nested dictionary in MongoDB)
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                update_fields[f"{key}.{sub_key}"] = sub_value
        else:
            update_fields[key] = value

    operations.append(
        UpdateOne(
            {"_id": ObjectId(item['id'])},  # corrected here
            {"$set": update_fields},
            upsert=True
        )
    )

# Execute the update operations in bulk
result = collection.bulk_write(operations)

# Calculate the total records updated
recordsUpdated = result.modified_count + len(result.upserted_ids)

# Output the results
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")


Total records: 8
Records updated: 0


**Description:**
Print out the results, showing the total number of records processed and the number of records updated.

In [72]:
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")

Total records: 8
Records updated: 0
